# 🎨 NeMo Data Designer: Synthetic Conversational Data with Person Details

> ⚠️ **Warning**: NeMo Data Designer is current in Early Release and is not recommended for production use.
>
> **Note**: In order to run this notebook, you must have the NeMo Data Designer microservice deployed locally via docker compose. See the [deployment guide](http://docs.nvidia.com/nemo/microservices/latest/set-up/deploy-as-microservices/data-designer/docker-compose.html) for more details.
>
> Alternatively, you can use the [NeMo Data Designer managed service](https://build.nvidia.com/nemo/data-designer). Please refer the [intro-tutorials](../../intro-tutorials/1-the-basics.ipynb) on how to connect to it. 
>
> **Note**: If you are using the NeMo Data Designer managed service, you will only be able to launch preview jobs. You will not be able to launch jobs using the `create` method.

This notebook demonstrates how to use the NeMo Data Designer to build a synthetic data generation pipeline step-by-step. We will create multi-turn user-assistant dialogues tailored for fine-tuning language models, enhanced with realistic person details. These synthetic dialogues can then be used as domain-specific training data to improve model performance in targeted scenarios.

These datasets could be used for developing and enhancing conversational AI applications, including customer support chatbots, virtual assistants, and interactive learning systems.

#### 💾 Install dependencies

**IMPORTANT** 👉 If you haven't already, follow the instructions in the [README](../../README.md) to install the necessary dependencies. Note you may need to restart your kernel after setting up the environment.

In [ ]:
from nemo_microservices.data_designer.essentials import (
    NeMoDataDesignerClient,
    CategorySamplerParams,
    DataDesignerConfigBuilder,
    LLMTextColumnConfig,
    SamplerColumnConfig,
    SamplerType,
    SubcategorySamplerParams,
    ModelConfig,
    InferenceParameters,
    LLMStructuredColumnConfig,
    Score,
    LLMJudgeColumnConfig
)

### ⚙️ Initialize the NeMo Data Designer Client

- The data designer client is responsible for submitting generation requests to the Data Designer microservice.
- In this notebook, we connect to a local deployment of data designer. You can deploy your own instance of data designer by following the deployment instructions [here](https://docs.nvidia.com/nemo/microservices/latest/set-up/deploy-as-microservices/data-designer/docker-compose.html).


In [ ]:
data_designer_client = NeMoDataDesignerClient(base_url="http://localhost:8080")

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- You must provide a list of model configs to the builder at initialization.

- This list contains the models you can choose from (via the `model_alias` argument) during the generation process.


In [ ]:
# We specify the endpoint of the model during deployment using the model_provider_registry.
model_id = "nvidia/nvidia-nemotron-nano-9b-v2"
model_alias = "nemotron-nano-9b-v2"

In [ ]:
config_builder = DataDesignerConfigBuilder(
    model_configs=[
        ModelConfig(
            alias=model_alias,
            provider="nvidiabuild",
            model=model_id,
            inference_parameters=InferenceParameters(
                max_tokens=1024,
                temperature=0.6,
                top_p=0.95,
            ),
        ),
    ]
)

### Define Pydantic Models for Structured Outputs

You can use Pydantic to define a structure for the messages that are produced by Data Designer

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field

class Message(BaseModel):
    """A single message turn in the conversation."""
    role: Literal["user", "assistant"] = Field(..., description="Which role is writing the message.")
    content: str = Field(..., description="Message contents.")


class ChatConversation(BaseModel):
    """A chat conversation between a specific user and an AI assistant.
    * All conversations are initiated by the user role.
    * The assistant role always responds to the user message.
    * Turns alternate between user and assistant roles.
    * The last message is always from the assistant role.
    * Message content can be long or short.
    * All assistant messages are faithful responses and must be answered fully.
    """
    conversation: list[Message] = Field(..., description="List of all messages in the conversation.")


class UserToxicityScore(BaseModel):
    """Output format for user toxicity assessment.

    Toxicity Scores:
    None: No toxicity detected in user messages.
    Mild: Slightly rude or sarcastic but not hateful or harmful.
    Moderate: Some disrespectful or harassing language.
    Severe: Overt hate, harassment, or harmful content.
    """
    reasons: list[str] = Field(..., description="Reasoning for user toxicity score.")
    score: Literal["None", "Mild", "Moderate", "Severe"] = Field(..., description="Level of toxicity observed in the user role responses.")

### 🌱 Adding Categorical Seed Columns

Define categorical seed columns that set the context for the generated dialogues. Domain, topic, complexity, conversation length, and user mood will influence the generated conversations.

In [ ]:
# Add domain column with subcategories for topics
config_builder.add_column(
    SamplerColumnConfig(
        name="domain",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["Tech Support", "Personal Finances", "Educational Guidance"]
        )
    )
)

# Add topic subcategory
config_builder.add_column(
    SamplerColumnConfig(
        name="topic",
        sampler_type=SamplerType.SUBCATEGORY,
        params=SubcategorySamplerParams(
            category="domain",
            values={
                "Tech Support": [
                    "Troubleshooting a Laptop",
                    "Setting Up a Home Wi-Fi Network",
                    "Installing Software Updates",
                ],
                "Personal Finances": [
                    "Budgeting Advice",
                    "Understanding Taxes",
                    "Investment Strategies",
                ],
                "Educational Guidance": [
                    "Choosing a College Major",
                    "Effective Studying Techniques",
                    "Learning a New Language",
                ],
            },
        )
    )
)

# Add complexity column
config_builder.add_column(
    SamplerColumnConfig(
        name="complexity",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["Basic", "Intermediate", "Advanced"]
        )
    )
)

# Add conversation length column
config_builder.add_column(
    SamplerColumnConfig(
        name="conversation_length",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=[2, 4, 6, 8]
        )
    )
)

# Add user mood column
config_builder.add_column(
    SamplerColumnConfig(
        name="user_mood",
        sampler_type=SamplerType.CATEGORY,
        params=CategorySamplerParams(
            values=["happy", "silly", "sarcastic", "combative", "disappointed", "toxic"]
        )
    )
)


### ✨ Adding Generated Data Columns
Now define the columns that the model will generate. These prompts instruct the LLM to produce the actual conversation: a system prompt to guide how the AI assistant engages in the conversation with the user, the conversation, and finally, we generate a toxicity_label to assess user toxicity over the entire conversation.

#### 💬🤖 AI Assistant system prompt and conversation

We generate a system prompt to base the AI assistant and then generate the entire conversation.

In [ ]:
# Generate assistant system prompt
config_builder.add_column(
    LLMTextColumnConfig(
        name="assistant_system_prompt",
        prompt="Write a reasonable system prompt for a helpful AI assistant with expertise in {{domain}} and {{topic}}. The AI assistant must not engage in harmful behaviors.",
        system_prompt="Keep this to a maximum of two sentences.",
        model_alias=model_alias,
    )
)

# Generate the user's task
config_builder.add_column(
    LLMTextColumnConfig(
        name="user_task",
        prompt="Define a simple task related to {{topic}} of {{complexity}} complexity for the user.",
        system_prompt="The task should be clear, focused on a single goal, and at most two sentences. Focus only on the task and don't provide only the task information.",
        model_alias=model_alias,
    )
)

# Generate the conversation
config_builder.add_column(
    LLMStructuredColumnConfig(
        name="conversation",
        prompt=(
            "<task>\n{{user_task}}\n</task>\n\n"
            "<system_prompt>{{assistant_system_prompt}}</system_prompt>\n\n"
            "Generate a conversation between a user and an AI assistant with <system_prompt> about <task>.\n"
            "User is asking the assistant for advice and is in a {{user_mood}} mood.\n"
            "The conversation must be {{conversation_length}} messages in length.\n"
            "The conversation must come to a natural end in {{conversation_length}} messages and if the assistant is unable "
            "to solve the user's needs by then, they should offer to continue the conversation later or redirect them to additional resources.\n"
            "The conversation must be realistic and natural, incorporating details from both the user's and assistant's profiles.\n"
            "The AI assistant role has the system prompt defined in <system_prompt>. All of its responses must conform to <system_prompt>:\n"
        ),
        output_format=ChatConversation,
        model_alias=model_alias,
    )
)


#### 🔍⚖️ Assessment Column: Toxicity Label
For the final step, we add a toxicity assessment column to classify the overall toxicity level of the user's messages across the entire conversation.

In [ ]:
# Toxicity Label for the entire conversation
# Define Toxicity Rubric
toxicity_rubric = Score(
    name="Toxicity",
    description="Evaluation of harmful, harassing, or hateful language in conversations",
    options=
        {
            "4": "No toxicity detected; conversation is respectful and appropriate.",
            "3": "Mild toxicity; slightly rude or sarcastic but not hateful or harmful.",
            "2": "Moderate toxicity; contains disrespectful or harassing language.",
            "1": "Severe toxicity; contains overt hate, harassment, or harmful content.",
            "0": "Extreme toxicity; contains dangerous, threatening, or abusive content."
        }
    
)

TOXICITY_PROMPT_TEMPLATE = """\
You are an expert in content moderation and digital communication ethics. You are evaluating the toxicity level in user messages.
Take a deep breath and carefully analyze the conversation provided. Use the rubric to score the level of toxicity in the USER'S messages only.

Consider:
- Focus primarily on harassment, hate, or harmful language
- Evaluate the full context of the conversation
- Be objective and consistent in your assessment
- If any single message exhibits toxicity, it should influence the final score

## CONVERSATION
{{ conversation }}
"""

config_builder.add_column(
    LLMJudgeColumnConfig(
        name="toxicity_evaluation",
        prompt=TOXICITY_PROMPT_TEMPLATE,
        scores=[toxicity_rubric],
        model_alias=model_alias
    )
)

## 👀 Generating a dataset preview

- Preview mode allows you to quickly iterate on your data design.

- Each preview generation call creates 10 records for inspection, helping you verify prompts and instructions before running a larger batch job.

In [ ]:
# Generate a preview
preview = data_designer_client.preview(config_builder)

In [ ]:
preview.display_sample_record()

## 🤔 Like what you see?

Submit a batch workflow!

In [ ]:
# # Submit batch job
job_results = data_designer_client.create(config_builder, num_records=20, wait_until_done=False)

job_results.wait_until_done()

In [ ]:
dataset = job_results.load_dataset()
print("\nGenerated dataset shape:", dataset.shape)

In [ ]:
# Inspect first 10 records of the generated dataset
dataset.head(10)